In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from collections import Counter
import statsmodels.api as sm
import matplotlib.font_manager as fm
from wordcloud import WordCloud
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import warnings; warnings.filterwarnings('ignore')
import tensorflow as tf
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import re
import os
import io
import time

plt.rc('font', family='NanumGothic')

In [23]:
#월간 할인 상품들을 크롤링해서 기업들이 월마다 어떤 추세로 할인을 진행하는지 알아보고자 함. 
#편의점, 마트 등 식품류에 한하여 알아보기. 
# pip install requests
# pip install beautifulsoup4
#위 식으로는 어디 편의점인지 알 수가 없어서 어디 편의점, 어디 카테고리인지 + 

import requests
from bs4 import BeautifulSoup

webpage = requests.get("https://pyony.com/search/")
soup = BeautifulSoup(webpage.content, "html.parser")

# 마지막 페이지 추출
page = soup.findAll("a", {"class":"page-link"})
last_page = page[len(page)-1].get('href')
_last_page = last_page.replace("?page=","")
    
print("last page", _last_page)

# 페이지별 크롤링
obj_result = []
category_result = []
for x in range(1, int(_last_page)):
    webpage = requests.get("https://pyony.com/search/?page="+str(x))
    soup = BeautifulSoup(webpage.content, "html.parser") 

                
    for j in range(0,20):
        cat = soup.findAll("div",{"class":"card-header"})[j].get_text().replace(" ", "").replace("\n", "|").split("|")
        _cat = []
        i= 0
        for cat_val in cat:
            if(cat_val):
                if(i < 4):
                    _cat.append(cat_val)
                    i = i+1
        category_result.append(_cat)
        obj = soup.findAll("div",{"class":"card-body"})[j].get_text().replace(" ", "").replace("\n", "|").split("|")
        _obj = []
        i = 0
        for val in obj:
            if(val):
                if(i < 4):
                    _obj.append(val)
                    i = i+1
        obj_result.append(_obj)


last page 345


In [99]:
obj_result

[['19)NAVIYA)콘돔돌출형3P', '5,500원', '(3,667원)', '2+1'],
 ['2023김토끼)슬림티슈(휴대용)', '2,500원', '(1,250원)', '1+1'],
 ['2080)더블액션초극세모', '2,000원', '(1,000원)', '1+1'],
 ['2080)바이오유산균요거민트125G', '3,900원', '(1,950원)', '1+1'],
 ['2080)바이오후레쉬요거민트치약125g', '3,900원', '(1,950원)', '1+1'],
 ['2080)센서티브미세모', '3,200원', '(1,600원)', '1+1'],
 ['2080)치약', '4,000원', '(2,000원)', '1+1'],
 ['2080)치약오리지날125g', '4,000원', '(2,000원)', '1+1'],
 ['2080)치약진지발리스오리지날125g', '3,900원', '(1,950원)', '1+1'],
 ['2080)칫솔', '2,800원', '(1,400원)', '1+1'],
 ['2080)칫솔더블액션플러스', '2,000원', '(1,000원)', '1+1'],
 ['2080)칫솔오리지날탄력모', '2,800원', '(1,400원)', '1+1'],
 ['2080)키즈가글250ml딸기맛(브레드)', '5,800원', '(2,900원)', '1+1'],
 ['2080)키즈가글250ml바나나맛(브레드)', '5,800원', '(2,900원)', '1+1'],
 ['23SS)남성모달팬티(105)', '9,900원', '(4,950원)', '1+1'],
 ['23SS)베이직쿨목토시', '5,900원', '(2,950원)', '1+1'],
 ['23SS)아쿠아엑스쿨토시블랙', '7,900원', '(3,950원)', '1+1'],
 ['23SS)아쿠아엑스쿨토시손등형블랙', '8,900원', '(4,450원)', '1+1'],
 ['23SS)아쿠아엑스쿨토시화이트', '7,900원', '(3,950원)', '1+1'],
 ['23SS)여성모달팬티(90)

In [100]:
category_result[0][:-1]

['GS25', '생활용품']

In [3]:
stores = []
for i in category_result:
        i = i[:-2]
        stores.append(i)

categories = []
for i in category_result:
        i = i[1:-1]
        categories.append(i)


obj = []
for i in obj_result:
        i= i[0:1]
        obj.append(i)


price = []
for i in obj_result:
        i= i[1:2]
        price.append(i)

discount = []
for i in obj_result:
        i= i[2:3]
        discount.append(i)

event = []
for i in obj_result:
        i= i[3:4]
        event.append(i)

In [38]:
(len(stores), len(categories),len(obj),len(price),len(discount), len(event))

(6880, 6880, 6880, 6880, 6880, 6880)

In [4]:
convenience_sales = pd.DataFrame(zip(stores, categories, obj, price, discount, event),
                                  columns=['stores', 'categories', 'obj', 'price', 'discount', 'event'])

In [5]:
convenience_sales

,stores,categories,obj,price,discount,event
0,[GS25],[생활용품],[19)NAVIYA)콘돔돌출형3P],"[5,500원]","[(3,667원)]",[2+1]
1,[emart24],[생활용품],[2023김토끼)슬림티슈(휴대용)],"[2,500원]","[(1,250원)]",[1+1]
2,[CU],[생활용품],[2080)더블액션초극세모],"[2,000원]","[(1,000원)]",[1+1]
3,[GS25],[생활용품],[2080)바이오유산균요거민트125G],"[3,900원]","[(1,950원)]",[1+1]
4,[emart24],[생활용품],[2080)바이오후레쉬요거민트치약125g],"[3,900원]","[(1,950원)]",[1+1]
...,...,...,...,...,...,...
6875,[GS25],[과자류],[흥선)아이스붐딸기맛1500],"[1,500원]","[(1,000원)]",[2+1]
6876,[GS25],[과자류],[흥선)아이스붐레몬맛1500],"[1,500원]","[(1,000원)]",[2+1]
6877,[GS25],[과자류],[흥선)아이스붐복숭아맛1500],"[1,500원]","[(1,000원)]",[2+1]
6878,[GS25],[과자류],[흥선)짱구미니믹스50G],"[2,500원]","[(1,250원)]",[1+1]


In [6]:
convenience_sales = convenience_sales.explode('stores')
convenience_sales = convenience_sales.explode('categories')
convenience_sales = convenience_sales.explode('obj')
convenience_sales = convenience_sales.explode('price')
convenience_sales = convenience_sales.explode('discount')
convenience_sales = convenience_sales.explode('event')

In [7]:
convenience_sales.discount =convenience_sales.discount.str.replace(',', '')

In [8]:
convenience_sales.price =convenience_sales.price.str.replace(',', '')

In [9]:
convenience_sales.discount =convenience_sales.discount.str.replace(')', '')
convenience_sales.discount =convenience_sales.discount.str.replace('(', '')

In [10]:
convenience_sales.price =convenience_sales.price.str.replace('원', '')
convenience_sales.discount =convenience_sales.discount.str.replace('원', '')

In [11]:
convenience_sales.price = convenience_sales.price.astype('int64')
convenience_sales.discount = convenience_sales.price.astype('int64')

In [13]:
convenience_sales.to_csv('./convenience_sales.csv', header=True, index=False)

In [19]:
convenience_sales.groupby("stores")['obj'].count()

stores
7-ELEVEn    1221
CU          1197
C·SPACE      479
GS25        1487
MINISTOP    1068
emart24     1428
Name: obj, dtype: int64

In [20]:
convenience_sales.groupby("categories").count()

,stores,obj,price,discount,event
categories,,,,,
과자류,1007,1007,1007,1007,1007
생활용품,989,989,989,989,989
식품,2029,2029,2029,2029,2029
아이스크림,568,568,568,568,568
음료,2287,2287,2287,2287,2287


In [21]:
convenience_sales.groupby(["stores","event"])["obj"].count()

stores    event
7-ELEVEn  1+1       290
          2+1       931
CU        1+1       326
          2+1       871
C·SPACE   1+1        66
          2+1       413
GS25      1+1       415
          2+1      1072
MINISTOP  1+1       294
          2+1       774
emart24   1+1       503
          2+1       925
Name: obj, dtype: int64

In [18]:
convenience_sales.groupby(["categories", "stores"])["obj"].count()

categories  stores  
과자류         7-ELEVEn    182
            CU          158
            C·SPACE      95
            GS25        200
            MINISTOP    171
            emart24     201
생활용품        7-ELEVEn    112
            CU          134
            C·SPACE      15
            GS25        309
            MINISTOP    114
            emart24     305
식품          7-ELEVEn    347
            CU          416
            C·SPACE      82
            GS25        432
            MINISTOP    341
            emart24     411
아이스크림       7-ELEVEn    157
            CU          127
            C·SPACE      77
            GS25         91
            MINISTOP      6
            emart24     110
음료          7-ELEVEn    423
            CU          362
            C·SPACE     210
            GS25        455
            MINISTOP    436
            emart24     401
Name: obj, dtype: int64

In [19]:
convenience_sales.groupby(["categories", "stores"])["price"].mean()

categories  stores  
과자류         7-ELEVEn     2190.934066
            CU           2117.088608
            C·SPACE      1931.578947
            GS25         2064.500000
            MINISTOP     2088.596491
            emart24      2298.507463
생활용품        7-ELEVEn     7138.392857
            CU           7133.955224
            C·SPACE      5003.333333
            GS25         6995.469256
            MINISTOP     7207.894737
            emart24      8347.704918
식품          7-ELEVEn     4018.299712
            CU           3970.024038
            C·SPACE      2868.292683
            GS25         4162.754630
            MINISTOP     4037.976540
            emart24      4075.182482
아이스크림       7-ELEVEn     7226.751592
            CU           4708.661417
            C·SPACE      7722.077922
            GS25         6307.692308
            MINISTOP    10533.333333
            emart24      5994.545455
음료          7-ELEVEn     2385.460993
            CU           2308.701657
            C·SPA

In [51]:
convenience_sales = pd.read_csv('./convenience_sales.csv', encoding='cp949')

In [49]:
convenience_sales = open('./convenience_sales.csv')

In [52]:
convenience_sales

,stores,categories,obj,price,discount,event
0,C·SPACE,식품,수입]마라곤약(위링스마라웨이)16g,500,500,1+1
1,C·SPACE,식품,수입]마라곤약(위링스향라웨이)16g,500,500,1+1
2,C·SPACE,생활용품,깨나라]여행용티슈50매(펭귄/곰),800,800,1+1
3,emart24,음료,청정)몽베스트330ml,800,800,1+1
4,C·SPACE,음료,청정]몽베스트생수330ml,800,800,1+1
...,...,...,...,...,...,...
6875,emart24,생활용품,닥터그루트)제이몬스터즈샴푸385ml_시트러스향,23000,23000,2+1
6876,7-ELEVEn,생활용품,P&G;)프로글라이드면도날2입,23200,23200,2+1
6877,MINISTOP,생활용품,P&amp;G)프로글라이드면도날2입,23200,23200,2+1
6878,emart24,생활용품,질레트)프로글라이드플렉스볼면도날2입,23200,23200,2+1


In [53]:
# ~를 사고싶다고 입력하면 ~~가서 사라고 입력해주는 것. 

product = input("사고싶은 물품을 입력하세요")
convenience_sales[convenience_sales['obj'].str.contains(product)]
    

,stores,categories,obj,price,discount,event
1031,emart24,생활용품,크리오)쿨맥스민트치약100g,3500,3500,1+1
1072,emart24,생활용품,2080)바이오후레쉬요거민트치약125g,3900,3900,1+1
1073,emart24,생활용품,2080)치약진지발리스오리지날125g,3900,3900,1+1
1074,CU,생활용품,46cm)쿨민트치약,3900,3900,1+1
1089,MINISTOP,생활용품,LG)페리오46cm치약,3900,3900,1+1
1090,7-ELEVEn,생활용품,LG)페리오46cm치약,3900,3900,1+1
1107,GS25,생활용품,애경)2080진지발리스K치약120G,3900,3900,1+1
1121,GS25,생활용품,페리오)46CM치약쿨민트100G,3900,3900,1+1
1122,emart24,생활용품,페리오)치약46cm100g,3900,3900,1+1
1124,CU,생활용품,2080)치약,4000,4000,1+1


In [ ]:
# ~를 사고싶다고 입력하면 ~~가서 사라고 입력해주는 것. 

product = input("사고싶은 물품을 입력하세요")
if product in convenience_sales[obj]:
    print("f{product}는 할인되고 있습니다.")

else: 
    print("할인항목에 없습니다. ")

In [ ]:
import requests
from bs4 import BeautifulSoup

webpage = requests.get("https://pyony.com/search/")
soup = BeautifulSoup(webpage.content, "html.parser")

# 마지막 페이지 추출
page = soup.findAll("a", {"class":"page-link"})
last_page = page[len(page)-1].get('href')
_last_page = last_page.replace("?page=","")
    
print("last page", _last_page)

# 페이지별 크롤링
obj_result = []
category_result = []
for x in range(1, int(_last_page)):
    webpage = requests.get("https://pyony.com/search/?page="+str(x))
    soup = BeautifulSoup(webpage.content, "html.parser") 

                
    for j in range(0,20):
        cat = soup.findAll("div",{"class":"card-header"})[j].get_text().replace(" ", "").replace("\n", "|").split("|")
        _cat = []
        i= 0
        for cat_val in cat:
            if(cat_val):
                if(i < 4):
                    _cat.append(cat_val)
                    i = i+1
        category_result.append(_cat)
        obj = soup.findAll("div",{"class":"card-body"})[j].get_text().replace(" ", "").replace("\n", "|").split("|")
        _obj = []
        i = 0
        for val in obj:
            if(val):
                if(i < 4):
                    _obj.append(val)
                    i = i+1
        obj_result.append(_obj)